### ETL USER_REVIEWS BASANDOME EN LAS FUNCIONES DADAS

In [1]:
#importo librerias

import pandas as pd
import numpy as np
import json
import glob
import ast
import gzip
import datetime

In [2]:
#ejecuto el archivo con un for in de gzip para que itere sobre cada linea del archivo comprimido ya que con pandas no se puede leer y pasarlo a dataframe
     #(no hizo falta descomprimirlo)

info = []    

for i in gzip.open('user_reviews.json.gz'): #itero cada linea
     info.append(ast.literal_eval(i.decode('utf-8')))       #le indico la codificacion y agrego cada diccionario a la lista vacia info                            
     
df_reviews = pd.DataFrame(info) #convierto a dataframe el resultado

exploracion_reviews = df_reviews.explode('reviews')     #uso json normalize para convertir la estructura anidada en un df, cada clave dicc lo convierte a columna.
reviews_normalizado = pd.json_normalize(exploracion_reviews['reviews'])                                                 
user_id = exploracion_reviews[['user_id']].reset_index(drop=True) #conservo la columna que deseo (en este caso user_id).
df_reviews = pd.concat([user_id, reviews_normalizado], axis=1) #concateno con pd.concat la columna user_id en el dataframe normalizado anteriormente.
df_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [3]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   funny        59305 non-null  object
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   item_id      59305 non-null  object
 5   helpful      59305 non-null  object
 6   recommend    59305 non-null  object
 7   review       59305 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


In [4]:
df_reviews.nunique()

user_id        25485
funny            186
posted          1906
last_edited     1015
item_id         3682
helpful         1344
recommend          2
review         55313
dtype: int64

In [5]:
df_reviews = df_reviews.dropna() #elimino valores nulos 

#### procedo a convertir la columna posted en formato fecha, ya que hay algunos valores a corregir

In [6]:
df_reviews['posted'] = df_reviews['posted'].str.extract(r'Posted ([\w\s\d,]+)') #extraigo la fecha del string

C:\Users\Valen\AppData\Local\Temp\ipykernel_12452\3008365801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['posted'] = df_reviews['posted'].str.extract(r'Posted ([\w\s\d,]+)') #extraigo la fecha del string


In [7]:
df_reviews['posted'] = df_reviews['posted'].replace({'Posted': ''}, regex=True)


C:\Users\Valen\AppData\Local\Temp\ipykernel_12452\13027061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['posted'] = df_reviews['posted'].replace({'Posted': ''}, regex=True)


In [8]:
df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], errors='coerce') #convierto la columna posted a un tipo datetime


C:\Users\Valen\AppData\Local\Temp\ipykernel_12452\835519235.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], errors='coerce') #convierto la columna posted a un tipo datetime


In [9]:
df_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,NaT,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,NaT,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,NaT,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,NaT,,730,No ratings yet,True,:D


In [10]:
df_reviews_final = df_reviews.copy()

In [11]:
df_reviews_final.to_parquet('df_reviews_final.parquet')